In [ ]:
!python -m pip install langchain langchain-core langchain-community langchain-experimental --quiet
!python -m pip install langchain-openai --quiet

!python -m pip install mlflow openai dspy litellm textstat evaluate --quiet
!python -m pip install --upgrade mlflow --quiet

%pip install --upgrade --quiet  yfinance

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool

/home/zadmin/Desktop/GAAI-B4-Azure/genai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
model_name='gpt4o'
model = AzureChatOpenAI(model=model_name)

In [3]:
import mlflow
mlflow.set_tracking_uri("http://20.75.92.162:5000/")

In [4]:
# Load the prompt
akshay_stock_code_generator = mlflow.genai.load_prompt("prompts:/akshay_stock_code_generator/1") # prompts:/<prompt-name>/<prompt-version>


akshay_sentiment_analyser = mlflow.genai.load_prompt("prompts:/akshay_sentiment_analyser/1") # prompts:/<prompt-name>/<prompt-version>


In [ ]:
# get the company stock code of the given company
company_name_prompt = ChatPromptTemplate([("system","For the given company name provide the stock code"), 
                                            ("user", akshay_stock_code_generator.template
                                          
                                            )])

ChatPromptTemplate(input_variables=['company_name'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='For the given company name provide the stock code'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['company_name'], input_types={}, partial_variables={}, template='what is the stock code for given company, \n                                              Provide only stock code\n                                              Strictly Donot provide any other details, If company name is not valid, ask valid.  \n                                              company name: {company_name}\n                                            '), additional_kwargs={})])

In [ ]:
# output parser
parser = StrOutputParser()

In [ ]:
# import the yahoo finance news tool api
news_tool = YahooFinanceNewsTool()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [17]:
sentiment_analyser_prompt = ChatPromptTemplate([("system","Extract the consise headlines from the news description " ), 
                                        ("user", akshay_sentiment_analyser.template)
                                        

])


In [22]:
chain = company_name_prompt | model | parser | news_tool | sentiment_analyser_prompt | model | parser
final_response = chain.invoke({"company_name":"Amazon"})
print(final_response)

[
    {
        "summary": "Morgan Stanley Maintains Overweight Rating on Amazon (AMZN) Amid Fresh Grocery Expansion",
        "sentiment": "Positive",
        "entities": {
            "people": [],
            "places": [],
            "companies": ["Morgan Stanley", "Amazon.com, Inc."]
        }
    }
]
